In [1]:
import pickle
import numpy as np

In [2]:
with open('embeddings_all.pkl', 'rb') as file:
    data = pickle.load(file)

x = data["outputs"].cpu().numpy()
y = data["labels"].cpu().numpy()
print(y.shape)

(49984,)


In [3]:
# normalize embeddings
x = (x - np.min(x)) / (np.max(x) - np.min(x))

In [4]:
embeddings = np.array(x)

In [6]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

def cluster_classes(embeddings, labels, n_clusters=10, classes_per_cluster=10):
    # Initialize clusters
    clusters = {i: [] for i in range(n_clusters)}
    unassigned_labels = set(labels)

    # Calculate and store all pairwise distances
    distances = euclidean_distances(embeddings, embeddings)

    # Precompute the minimum distance to each unassigned label from each cluster
    min_distances = {label: np.inf for label in unassigned_labels}

    for cluster_id in range(n_clusters):
        for _ in range(classes_per_cluster):
            if cluster_id == 0:
                # For the first cluster, choose classes randomly
                chosen_label = np.random.choice(list(unassigned_labels))
            else:
                # For subsequent clusters, choose based on minimum cumulative distance
                chosen_label = min(unassigned_labels, key=lambda label: min_distances[label])

            # Assign chosen label to current cluster and remove from unassigned labels
            clusters[cluster_id].append(chosen_label)
            unassigned_labels.remove(chosen_label)

            # Update minimum distances for the remaining unassigned labels
            for label in unassigned_labels:
                min_distances[label] = min(min_distances[label], distances[chosen_label, label])

    return clusters
# Example usage
# embeddings = np.array([...])  # Replace with your embeddings array
# labels = np.array([...])      # Replace with your labels array
# result = cluster_classes(embeddings, labels)
# print(result)

In [7]:
result = cluster_classes(embeddings, y)